In [68]:
import os
import pandas as pd
import transformers

In [54]:
train_path = os.path.join("data","en_partut-ud-train.conllu")
dev_path = os.path.join("data","en_partut-ud-dev.conllu")
test_path = os.path.join("data","en_partut-ud-test.conllu")

In [63]:
HEADER_CONST = "# sent_id = "
TEXT_CONST = "# text = "
STOP_CONST = "\n"
WORD_OFFSET = 1
LABEL_OFFSET = 3


def txt_to_dataframe(data_path):
    '''
    read UD text file and convert to df format
    '''
    with open(data_path, "r") as fp:
        df = pd.DataFrame(
            columns={
                "text",
                "word",
                "label"
            }
        )
        for line in fp.readlines():
            if TEXT_CONST in line:
                words_list = []
                labels_list = []
                text = line.split(TEXT_CONST)[1]
                # this is a new text, need to parse all the words in it
            elif line is not STOP_CONST and HEADER_CONST not in line:
                temp_list = line.split("\t")
                words_list.append(temp_list[WORD_OFFSET])
                labels_list.append(temp_list[LABEL_OFFSET])
            if line == STOP_CONST:
                # this is the end of the text, adding to df
                cur_df = pd.DataFrame(
                    {
                        "text": len(words_list) * [text],
                        "word": words_list,
                        "label": labels_list
                    }
                )
                df = pd.concat([df,cur_df])
        return df
            


In [64]:
df_train = txt_to_dataframe(train_path)
df_dev = txt_to_dataframe(dev_path)
df_test = txt_to_dataframe(test_path)

In [65]:
df_train["label"].value_counts()

NOUN     9249
ADP      5220
PUNCT    5105
DET      4616
VERB     4126
ADJ      3410
AUX      2076
PROPN    2033
PRON     1734
ADV      1707
CCONJ    1472
PART     1168
NUM       787
SCONJ     627
X         140
SYM        42
_          27
INTJ        6
Name: label, dtype: int64

In [66]:
df_dev["label"].value_counts()

NOUN     568
PUNCT    353
ADP      297
VERB     276
DET      266
ADJ      210
PRON     153
AUX      124
ADV      108
PROPN    107
CCONJ     88
NUM       60
PART      56
SCONJ     41
X         13
SYM        2
_          1
Name: label, dtype: int64

In [67]:
df_test["label"].value_counts()

NOUN     753
ADP      488
DET      439
PUNCT    339
VERB     326
AUX      234
ADJ      224
ADV      131
PRON     106
CCONJ     96
PROPN     90
PART      66
NUM       61
SCONJ     51
_          4
X          2
INTJ       2
Name: label, dtype: int64

In [73]:
df_train["text"].apply(lambda x: "[CLS] " + x.lower().replace("\n",""))

0    [CLS] distribution of this license does not cr...
1    [CLS] distribution of this license does not cr...
2    [CLS] distribution of this license does not cr...
3    [CLS] distribution of this license does not cr...
4    [CLS] distribution of this license does not cr...
                           ...                        
5    [CLS] no shakespearean poems were included in ...
6    [CLS] no shakespearean poems were included in ...
7    [CLS] no shakespearean poems were included in ...
8    [CLS] no shakespearean poems were included in ...
9    [CLS] no shakespearean poems were included in ...
Name: text, Length: 43545, dtype: object

In [74]:
BERT_BASE_MODEL = "bert-base-uncased"

model = transformers.BertModel.from_pretrained(BERT_BASE_MODEL)
tokenizer = transformers.BertTokenizer.from_pretrained(BERT_BASE_MODEL)